In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import mysql.connector
import time

In [2]:
# Connect to MySQL database
mydb = mysql.connector.connect(
    host="jobs-scraping.cux1s0fa60hj.us-east-2.rds.amazonaws.com",
    user="root",
    password="123456789",
    database="job_scraping_database"
)

cursor = mydb.cursor()


In [3]:
# Define chrome_options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("executable_path=./chromedriver.exe")
chrome_options.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"

In [4]:
wd = webdriver.Chrome(options = chrome_options)

In [5]:
wd.get("https://www.careerbuilder.com/jobs?cb_apply=false&cb_workhome=all&emp=all&keywords=Marketing+Entry+Level+&location=United+States&page_number=1")
time.sleep(5)

In [6]:
max_retries = 3

# Locate the left side of the job listing area (replace with correct selector)
job_listing_left_elem = wd.find_element(By.ID, "col-fixed")

while True:
    try:
        # Scroll to the bottom of the job listing left side
        wd.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", job_listing_left_elem)
        time.sleep(2)  # Adjust sleep time as needed
        
        # Find and click the "Load More" button (if present)
        try:
            load_more = job_listing_left_elem.find_element(By.XPATH, "//button[@class='btn btn-clear btn-clear-blue b-i']")
            for retry_attempt in range(max_retries):
                try:
                    # Scroll to the element before clicking
                    wd.execute_script("arguments[0].scrollIntoView();", load_more)
                    time.sleep(3)
                    
                    # Click the element
                    load_more.click()
                    time.sleep(3)
                    
                    # Break the retry loop if the click is successful
                    break
                except ElementClickInterceptedException:
                    # Wait for a brief moment before retrying
                    time.sleep(2)
                    
                    if retry_attempt == max_retries - 1:
                        # End the loop if maximum retries reached
                        print("1 No more data to load.")
                        break
        except NoSuchElementException:
            print("2 No more data to load.")
            break
    
    except NoSuchElementException:
        print("No more data")
        break
    except ElementNotInteractableException:
        print("3 No more data to load.")
        break

1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
1 No more data to load.
3 No more data to load.


In [7]:
jobs_src = wd.page_source
soup = bs(jobs_src, "html.parser")
target_classes = ["data-results-content-parent relative", "data-results-content-parent relative bg-fade"]
jobs = soup.find_all("li", class_ = target_classes)
for job in jobs:
    # JobId
    job_id = job.find("a", class_ = "data-results-content block job-listing-item")["data-job-did"]
    
    #Check if the JobId already exists in the table
    select_query = "SELECT JobId FROM marketing_jobs WHERE JobId = %s"
    cursor.execute(select_query, (job_id,))
    result = cursor.fetchone()
    
    if result is None:
        #Job Title
        job_title = job.find("div", class_ = "data-results-title dark-blue-text b").text.strip()
        
        #Job Link
        job_link = "https://www.careerbuilder.com" + job.find("a", class_ = "data-results-content block job-listing-item")["href"]
        
        data_detail = job.find("div", class_ = "data-details").find_all("span")
    
        # Company Name + Location
        if len(data_detail) == 3:
            company_name = data_detail[0].text.strip()
            location = data_detail[1].text.strip()
        else:
            company_name = ""
            location = data_detail[0].text.strip()

        driver = webdriver.Chrome(options = chrome_options)
        driver.get(job_link)
        time.sleep(3)

        job_src = driver.page_source
        job_soup = bs(job_src, "html.parser")

        # Salary
        salary_elem = job_soup.find("div", id = "cb-salcom-info")
        salary = salary_elem.text.strip() if salary_elem else ""
        if ":" in salary:
            salary = salary.split(":")[1].strip()

        # Job Description
        job_des_elem = job_soup.find("div", id = "jdp_description")
        job_des = job_des_elem.text.strip() if job_des_elem else ""

        #Skill
        list_skill = ""
        skill_elem = job_soup.find("ul", class_ = "pl0 no-marker")
        skill_list = []
        if skill_elem:
            skills = skill_elem.find_all("li")
            for skill in skills:
                skill_list.append(skill.text.strip())
            list_skill = ", ".join(skill_list)
        else:
            list_skill = ""
        
        #Insert the data into the MySQL table
        insert_query = '''
            INSERT INTO marketing_jobs (JobId, Title, Company, Location, Apply, JobDescription, Skills, Salary)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        '''
        insert_values = (job_id, job_title, company_name, location, job_link, job_des, list_skill, salary)
        cursor.execute(insert_query, insert_values)
        mydb.commit()
        driver.quit()
    else:
        print("Skipping duplicate JobId:", job_id)

Skipping duplicate JobId: JD834M60MDXHF7MJM6X
Skipping duplicate JobId: J8S43Z74PD2Q81DJ2SY
Skipping duplicate JobId: J312JD77CJ88TND01QQ
Skipping duplicate JobId: JDD63T67BT372W3SF1W
Skipping duplicate JobId: JCL0G66BS1L7RGRYGWF
Skipping duplicate JobId: J2W2HD74V6WCCY987H1
Skipping duplicate JobId: J2N86667KTW5SV5JQG4
Skipping duplicate JobId: J2S63N65K553R3HNMG6
Skipping duplicate JobId: J2W6RS79NZLKSW0BGSJ
Skipping duplicate JobId: J2W8FS78QK7DL1F4RQ1
Skipping duplicate JobId: J337M26YW57TH4GJ9RS
Skipping duplicate JobId: JD66745W9ZX6RH5FPTY
Skipping duplicate JobId: J2S5NM6M16ZRCBFNX23
Skipping duplicate JobId: J3036K6VN5K6W2FJ4G8
Skipping duplicate JobId: JD97WY77ZP8GT6GGTWR
Skipping duplicate JobId: J2N7WL5XNSB21391903
Skipping duplicate JobId: J314JX6P9J7T3GNFKFP
Skipping duplicate JobId: JDF3SN6JF7DDBZ5RW9V
Skipping duplicate JobId: JCL6D66398S08M3ZDS0
Skipping duplicate JobId: J2W2ZQ5WZ9B5G421BNZ
Skipping duplicate JobId: JDF4R95Y1SFSMMPTSXR
Skipping duplicate JobId: J2W5P56Y